In [ ]:
#@title ### Install the Graph Nets library on this Colaboratory runtime  { form-width: "60%", run: "auto"}
#@markdown <br>1. Connect to a local or hosted Colaboratory runtime by clicking the **Connect** button at the top-right.<br>2. Choose "Yes" below to install the Graph Nets library on the runtime machine with:<br>

install_graph_nets_library = "Yes"  #@param ["Yes", "No"]
install_pybrite = "Yes"  #@param ["Yes", "No"]
install_tf_gpu = "No"  #@param ["Yes", "No"]

print("Installing Tensorflow library with:")
print("  $ pip install tensorflow=='r1.14'\n")
print("Output message from command:\n")
#!pip uninstall tensorflow tensorflow-gpu
!pip install tensorflow=="1.14.0"

if install_graph_nets_library.lower() == "yes":
    print("Installing variation of Graph Nets library from github with:")
    print("  $ git clone https://github.com/caiodadauto/graph_nets\n")
    print("  $ pip install graph_nets/\n")
    print("Output message from command:\n")
    !git clone https://github.com/caiodadauto/graph_nets
    !pip install graph_nets/
else:
    print("Skipping installation of Graph Nets library")
    
if install_tf_gpu.lower() == "yes":
    print("Installing Tensorflow GPU library with:")
    print("  $ pip install tensorflow-gpu\n")
    print("Output message from command:\n")
    !pip install tensorflow-gpu
else:
    print("Skipping installation of Tensorflow GPU library")

if install_pybrite.lower() == "yes":
    print("Installing pybrite from github with:")
    print("  $ git clone --single-branch --branch colab https://github.com/caiodadauto/pybrite.git\n")
    print("  $ pip install pybrite/pybrite/")
    print("Output message from command:\n")
    !git clone --single-branch --branch colab https://github.com/caiodadauto/pybrite.git
    !pip install pybrite/pybrite/
else:
    print("Skipping installation of Pybrite library")

In [ ]:
#@title Imports  { form-width: "30%" }
import os
import time

import numpy as np
import pandas as pd
import networkx as nx
import sonnet as snt
import tensorflow as tf
import matplotlib.pyplot as plt

import pybrite
from graph_nets import graphs
from graph_nets import blocks
from graph_nets import modules
from graph_nets import utils_np
from graph_nets import utils_tf

In [ ]:
#@title Set google drive access  { form-width: "30%" }

from google.colab import drive
drive.mount('/content/gdrive')

In [1]:
#@title Set Seed, Layers Number and their Size { form-width: "30%" }

SEED = 2  #@param{type: 'integer'}
NUM_LAYERS = 2  #@param{type: 'integer'}
LATENT_SIZE = 16  #@param{type: 'integer'}
TEST_LOCAL_STATS = False
SCALE = True
DRIVE_PATH = "/content/gdrive/My Drive/"

In [ ]:
#@title Helper Functions: Tensorflow Integration { form-width: "30%" }

def create_placeholders(batch_generator):
    # Create some example data for inspecting the vector sizes.
    input_graphs, target_graphs, _ = next(batch_generator)
    input_ph = utils_tf.placeholders_from_networkxs(input_graphs)
    target_ph = utils_tf.placeholders_from_networkxs(target_graphs)
    
    dtype = tf.as_dtype(utils_np.networkxs_to_graphs_tuple(target_graphs).edges.dtype)
    weight_ph = tf.placeholder(dtype, name="loss_weights")
    is_training_ph = tf.placeholder(tf.bool, name="training_flag")
    return input_ph, target_ph, weight_ph, is_training_ph

def create_feed_dict(batch_generator, is_training, weights, input_ph, target_ph, is_training_ph, weight_ph):
    inputs, targets, pos = next(batch_generator)
    input_graphs = utils_np.networkxs_to_graphs_tuple(inputs)
    target_graphs = utils_np.networkxs_to_graphs_tuple(targets)
    
    if weights[0] != 1 or weights[1] != 1:
        batch_weights = np.ones(target_graphs.edges.shape[0])
        target_args = np.argmax(target_graphs.edges, axis=-1)
        batch_weights[target_args == 0] *= weights[0]
        batch_weights[target_args == 1] *= weights[1]
    else:
        batch_weights = 1
    
    feed_dict = {input_ph: input_graphs, target_ph: target_graphs, is_training_ph: is_training, weight_ph: batch_weights}
    return feed_dict, pos

def compute_accuracy(target, output, distribution=False):
    acc_all = []
    solved_all = []
    acc_true_all = []
    acc_false_all = []
    solved_true_all = []
    solved_false_all = []
    
    tg_dict = utils_np.graphs_tuple_to_data_dicts(target)
    out_dict = utils_np.graphs_tuple_to_data_dicts(output)
    for tg_graph, out_graph in zip(tg_dict, out_dict):
        expect = np.argmax(tg_graph["edges"], axis=-1)
        predict = np.argmax(out_graph["edges"], axis=-1)
        true_mask = np.ma.masked_equal(expect, 1).mask
        false_mask = np.ma.masked_equal(expect, 0).mask

        acc = (expect == predict)
        acc_true = acc[true_mask]
        acc_false = acc[false_mask]

        solved = np.all(acc)
        solved_true = np.all(acc_true)
        solved_false = np.all(acc_false)

        acc_all.append(np.mean(acc))
        acc_true_all.append(np.mean(acc_true))
        acc_false_all.append(np.mean(acc_false))
        
        solved_all.append(solved)
        solved_true_all.append(solved_true)
        solved_false_all.append(solved_false)
    acc_all = np.stack(acc_all)
    acc_true_all = np.stack(acc_true_all)
    acc_false_all = np.stack(acc_false_all)

    solved_all = np.stack(solved_all)
    solved_true_all = np.stack(solved_true_all)
    solved_false_all = np.stack(solved_false_all)
    if not distribution:
        acc_all = np.mean(acc_all)
        acc_true_all = np.mean(acc_true_all)
        acc_false_all = np.mean(acc_false_all)

        solved_all = np.mean(solved_all)
        solved_true_all = np.mean(solved_true_all)
        solved_false_all = np.mean(solved_false_all)
    return acc_all, solved_all, acc_true_all, solved_true_all, acc_false_all, solved_false_all

def get_generator_path_metrics(inputs, targets, outputs):
    out_dicts = utils_np.graphs_tuple_to_data_dicts(outputs)
    in_dicts = utils_np.graphs_tuple_to_data_dicts(inputs)
    tg_dicts = utils_np.graphs_tuple_to_data_dicts(targets)

    def softmax_prob(x):  # pylint: disable=redefined-outer-name
        e = np.exp(x)
        return e / np.sum(e, axis=-1, keepdims=True)
    
    n_graphs = len(tg_dicts)
    for tg_graph, out_graph, in_graph, idx_graph in zip(tg_dicts, out_dicts, in_dicts, range(n_graphs)):
        n_node = out_graph["n_node"]
        tg_graph_dist = tg_graph["nodes"][:,0]
        tg_graph_hops = tg_graph["nodes"][:,1]
        out_graph_dist = np.zeros_like(tg_graph_dist)
        out_graph_hops = np.zeros_like(tg_graph_dist)
        end_node = np.argwhere(tg_graph_dist == 0).reshape(1)[0]
        for node in range(n_node):
            hops = 0
            strength = 0
            start = node
            sender = None
            reachable = True
            path = np.zeros(n_node, dtype=np.bool)
            while start != end_node:
                path[start] = True
                start_edges_idx = np.argwhere(out_graph["senders"] == start).reshape(-1,)
                receivers = out_graph["receivers"][start_edges_idx]
                start_edges = out_graph["edges"][start_edges_idx]
                edges_prob = softmax_prob(start_edges)
                
                remove_sender = (receivers != sender) if sender else np.ones_like(receivers, dtype=np.bool)
                routing_links = edges_prob[remove_sender, 0] < edges_prob[remove_sender, 1]

                if not np.any(routing_links):
                    routing_links = ~routing_links
                    edge_forward_idx = np.argmin(edges_prob[remove_sender, 0] - edges_prob[remove_sender, 1])
                else:
                    edge_forward_idx = np.argmax(edges_prob[remove_sender][routing_links][:, -1])

                if edge_forward_idx.size > 1:
                    print("\nMore than one max prob\n")

                sender = start
                start = receivers[remove_sender][routing_links][edge_forward_idx]
                
                if path[start]:
                    reachable = False
                    break
                    
                hops += 1
                strength += in_graph["edges"][start_edges_idx][remove_sender][routing_links][edge_forward_idx][0]
            if reachable:
                out_graph_dist[node] = strength
                out_graph_hops[node] = hops
        out_graph_hops = np.delete(out_graph_hops, end_node)
        out_graph_dist = np.delete(out_graph_dist, end_node)
        tg_graph_hops = np.delete(tg_graph_hops, end_node)
        tg_graph_dist = np.delete(tg_graph_dist, end_node)
        idx_non_zero = np.flatnonzero(out_graph_hops)
        unreachable_p =  1 - idx_non_zero.size / out_graph_dist.size
        if idx_non_zero.size > 0:
            diff_dist = (np.abs(out_graph_dist[idx_non_zero] - tg_graph_dist[idx_non_zero]))
            diff_hops = (np.abs(out_graph_hops[idx_non_zero] - tg_graph_hops[idx_non_zero]))
            yield (diff_dist, diff_hops, unreachable_p)
        else:
            yield (None, None, unreachable_p)

def aggregator_path_metrics(inputs, targets, outputs, distribution=False):
    n_graphs = targets.n_node.size
    idx_graph = 0
    none_idx = []
    hist_hops = []
    hist_dist = []
    batch_max_dist_diff = np.zeros(n_graphs)
    batch_min_dist_diff = np.zeros(n_graphs)
    batch_avg_dist_diff = np.zeros(n_graphs)
    batch_max_hops_diff = np.zeros(n_graphs)
    batch_min_hops_diff = np.zeros(n_graphs)
    batch_avg_hops_diff = np.zeros(n_graphs)
    batch_unreachable_p = np.zeros(n_graphs)
    metrics_graph_generator = get_generator_path_metrics(inputs, targets, outputs)
    for diff_dist, diff_hops, unreachable_p in metrics_graph_generator:
        batch_unreachable_p[idx_graph] = unreachable_p
        
        if np.any(diff_dist == None):
            none_idx.append(idx_graph)
        else:
            batch_max_dist_diff[idx_graph] = np.max(diff_dist)
            batch_min_dist_diff[idx_graph] = np.min(diff_dist)
            batch_avg_dist_diff[idx_graph] = np.mean(diff_dist)
            batch_max_hops_diff[idx_graph] = np.max(diff_hops)
            batch_min_hops_diff[idx_graph] = np.min(diff_hops)
            batch_avg_hops_diff[idx_graph] = np.mean(diff_hops)
            if distribution:
                hist_hops.append(diff_hops)
                hist_dist.append(diff_dist)
        idx_graph += 1
    batch_max_dist_diff = np.delete(batch_max_dist_diff, none_idx)
    batch_min_dist_diff = np.delete(batch_min_dist_diff, none_idx)
    batch_avg_dist_diff = np.delete(batch_avg_dist_diff, none_idx)
    batch_max_hops_diff = np.delete(batch_max_hops_diff, none_idx)
    batch_min_hops_diff = np.delete(batch_min_hops_diff, none_idx)
    batch_avg_hops_diff = np.delete(batch_avg_hops_diff, none_idx)
    if not distribution:
        return dict(avg_batch_max_dist_diff=np.mean(batch_max_dist_diff) if batch_max_dist_diff.size else np.infty,
                    avg_batch_min_dist_diff=np.mean(batch_min_dist_diff) if batch_min_dist_diff.size else np.infty,
                    avg_batch_avg_dist_diff=np.mean(batch_avg_dist_diff) if batch_avg_dist_diff.size else np.infty,
                    avg_batch_max_hops_diff=np.mean(batch_max_hops_diff) if batch_max_hops_diff.size else np.infty,
                    avg_batch_min_hops_diff=np.mean(batch_min_hops_diff) if batch_min_hops_diff.size else np.infty,
                    avg_batch_avg_hops_diff=np.mean(batch_avg_hops_diff) if batch_avg_hops_diff.size else np.infty,
                    max_batch_unreachable_p=np.max(batch_unreachable_p),
                    min_batch_unreachable_p=np.min(batch_unreachable_p),
                    avg_batch_unreachable_p=np.mean(batch_unreachable_p))
    else:
        return {"percentage of unreachable paths":batch_unreachable_p, "difference of hops":np.concatenate(hist_hops), "difference of strength":np.concatenate(hist_dist)}

def create_loss_ops(target_op, output_ops, weight):
    loss_ops = [
        tf.losses.softmax_cross_entropy(target_op.edges, output_op.edges, weights=weight)
        for output_op in output_ops
    ]
    return loss_ops

def make_all_runnable_in_session(*args):
    """Lets an iterable of TF graphs be output from a session as NP graphs."""
    return [utils_tf.make_runnable_in_session(a) for a in args]

In [ ]:
#@title Helper Functions: Create Layers { form-width: "30%" }

class LeakyReluMLP(snt.AbstractModule):
    def __init__(self,
                 hidden_size,
                 n_layers,
                 name="LeakyReluNormMLP"):
        super(LeakyReluMLP, self).__init__(name=name)
        self._n_layers = n_layers
        self._hidden_size = hidden_size
        with self._enter_variable_scope():
            self._linear_layers = []
            for _ in range(self._n_layers):
                self._linear_layers.append(snt.Linear(self._hidden_size))
            
    def _build(self, inputs, is_training):
        outputs_op = inputs
        for linear in self._linear_layers:
            outputs_op = linear(outputs_op)
            outputs_op = tf.nn.leaky_relu(outputs_op, alpha=0.05)
        return outputs_op


class LeakyReluNormMLP(snt.AbstractModule):
    def __init__(self,
                 hidden_size,
                 n_layers,
                 name="LeakyReluNormMLP"):
        super(LeakyReluNormMLP, self).__init__(name=name)
        self._n_layers = n_layers
        self._hidden_size = hidden_size
        with self._enter_variable_scope():
            self._linear_layers = []
            self._bn_layers = []
            for _ in range(self._n_layers):
                self._linear_layers.append(snt.Linear(self._hidden_size))
                self._bn_layers.append(snt.BatchNorm(scale=SCALE))
            
    def _build(self, inputs, is_training):
        outputs_op = inputs
        for linear, bn in zip(self._linear_layers, self._bn_layers):
            outputs_op = linear(outputs_op)
            outputs_op = tf.nn.leaky_relu(outputs_op, alpha=0.05)
            outputs_op = bn(outputs_op, is_training=is_training, test_local_stats=TEST_LOCAL_STATS)
        return outputs_op
    
class LeakyReluNormGRU(snt.AbstractModule):
    def __init__(self,
                 hidden_size,
                 recurrent_dropout=0.75,
                 name="LeakyReluNormGRU"):
        super(LeakyReluNormGRU, self).__init__(name=name)
        self._hidden_size = hidden_size
        with self._enter_variable_scope():
            self._gru = snt.GRU(self._hidden_size)
            self._dropout_gru = snt.python.modules.gated_rnn.RecurrentDropoutWrapper(self._gru, recurrent_dropout)
            self._batch_norm = snt.BatchNorm(scale=SCALE)
    
    def get_initial_state(self, batch_size, dtype=tf.float64):
        return self._dropout_gru.initial_state(batch_size, dtype=dtype)
    
    def _build(self, inputs, prev_states, is_training):
        def true_fn():
            return self._dropout_gru(inputs, prev_states)
        
        def false_fn():
            o, ns = self._gru(inputs, prev_states[0])
            ns = (ns, [tf.ones_like(ns, name="FoolMask")])
            return o, ns
        
        outputs_op, next_states = tf.cond(is_training, true_fn=true_fn, false_fn=false_fn)
        outputs_op = tf.nn.leaky_relu(outputs_op, alpha=0.05)
        outputs_op = self._batch_norm(outputs_op, is_training=is_training, test_local_stats=TEST_LOCAL_STATS)
        return outputs_op, next_states

def make_gru_model(size=LATENT_SIZE):
    """Instantiates a new MLP, followed by LayerNorm.

    The parameters of each new MLP are not shared with others generated by
    this function.

    Returns:
    A Sonnet module which contains the MLP and LayerNorm.
    """
    return LeakyReluNormGRU(size)

def make_mlp_model(size=LATENT_SIZE, n_layers=NUM_LAYERS, model=LeakyReluNormMLP):
    """Instantiates a new MLP, followed by LayerNorm.

    The parameters of each new MLP are not shared with others generated by
    this function.

    Returns:
    A Sonnet module which contains the MLP and LayerNorm.
    """
    return model(size, n_layers)

In [ ]:
#@title Helper Classes: Modules to Integrate the Encode-Process-Decode Architecture { form-width: "30%" }

class LocalRoutingNetwork(snt.AbstractModule):
    """Implement neural network to deal with local routing table lookup.
    
    See net.in.tum.de/fileadmin/bibtex/publications/papers/geyer2018bigdama.pdf
    figure 2 for more details.
    """

    def __init__(self,
                 output_size,
                 query_model_fn=make_mlp_model,
                 weight_model_fn=make_mlp_model,
                 n_heads=3,
                 name="LocalRoutingNetwork"):
        super(LocalRoutingNetwork, self).__init__(name=name)
        #self._multihead_weight = []
        with self._enter_variable_scope():
            self._query_model = query_model_fn()
            self._logist_routing = snt.Linear(output_size, name="routing_logist_layer")
            #for _ in range(n_heads):
            #    self._multihead_weight.append(weight_model_fn())
            self._model_weight = weight_model_fn()
        
    def _build(self, inputs, **kwargs):
        query_output = self._query_model(inputs.globals, **kwargs)
        queries_output = utils_tf.repeat(query_output, inputs.n_edge)
        point_wise = tf.multiply(queries_output, inputs.edges)
        senders_feature = tf.gather(inputs.nodes, inputs.senders)
        weight_input = tf.concat([senders_feature, point_wise], -1)
        #weights = []
        #for model in self._multihead_weight:
        #    weight_output = self._logist_routing(model(weight_input, **kwargs))
        #    weights.append(self._unsorted_segment_softmax(
        #        weight_output, inputs.senders, tf.reduce_sum(inputs.n_node)))
        #output_edges = tf.reduce_mean(tf.stack(weights), axis=0)
        output_edges = self._logist_routing(self._model_weight(weight_input, **kwargs))
        return inputs.replace(edges=output_edges)
    
    def _unsorted_segment_softmax(self, x, idx, n_idx):
        op1 = tf.exp(x)
        op2 = tf.unsorted_segment_sum(op1, idx, n_idx)
        op3 = tf.reduce_sum(op2, -1, keepdims=True)
        op4 = tf.gather(op3, idx)
        op5 = tf.divide(op1, op4)
        return op5
    
class MLPGraphIndependent(snt.AbstractModule):
    """GraphIndependent with MLP edge, node, and global models."""

    def __init__(self, name="MLPGraphIndependent"):
        super(MLPGraphIndependent, self).__init__(name=name)
        with self._enter_variable_scope():
            self._network = modules.GraphIndependent(
                edge_model_fn=make_mlp_model,
                node_model_fn=make_mlp_model,
                global_model_fn=None)

    def _build(self, inputs, **kwargs):
        return self._network(inputs, **kwargs)

class GraphGatedNonLocalNetwork(snt.AbstractModule):
    """Implementation of Non-Local Neural Network. Basically, there is not used
    global features, only ones of the nodes and edges.

    See arxiv.org/abs/1806.01261 Figura 4d for more deatais about network,
    beyond there is made the update on edge's features.
    """

    def __init__(self,
                 gate_recurrent_model_fn=make_gru_model,
                 bias_shape=[LATENT_SIZE * 3],
                 reducer=tf.unsorted_segment_sum,
                 name="GraphGatedNonLocalNetwork"):
        """Initializes the GraphGatedNonLocalNetwork module.

        Args:
            edge_model_fn: A callable that will be passed to EdgeBlock to perform
                per-edge computations. The callable must return a Sonnet module (or
                equivalent; see EdgeBlock for details).
            node_model_fn: A callable that will be passed to NodeBlock to perform
                per-node computations. The callable must return a Sonnet module (or
                equivalent; see NodeBlock for details).
            reducer: Reducer to be used by NodeBlock to aggregate nodes and edges.
                Defaults to tf.unsorted_segment_sum.
            name: The module name.
        """
        super(GraphGatedNonLocalNetwork, self).__init__(name=name)

        with self._enter_variable_scope():
            self._edge_block = blocks.GatedEdgeBlock(
                gate_recurrent_model_fn=gate_recurrent_model_fn,
                use_edges=True,
                use_receiver_nodes=True,
                use_sender_nodes=True,
                use_globals=False
            )
            self._node_block = blocks.GatedNodeBlock(
                gate_recurrent_model_fn=gate_recurrent_model_fn,
                bias_shape=bias_shape,
                use_received_edges=True,
                use_sent_edges=False,
                use_nodes=True,
                use_globals=False
            )

    def reset_state(self, edge_batch_size,  node_batch_size, edge_state=None, node_state=None):
        self._edge_block.reset_state(edge_batch_size, state=edge_state)
        self._node_block.reset_state(node_batch_size, state=node_state)
            
    def _build(self, graph, **kwargs):
        """Connects the GraphGatedNonLocalNetwork.

        Args:
          graph: A `graphs.GraphsTuple` containing `Tensor`s. The features of
            each nodes and edges of `graph` should be concatenable on the last dimension.

        Returns:
          An output `graphs.GraphsTuple` with updated edges, nodes and globals.
        """
        
        return self._node_block(self._edge_block(graph, **kwargs), **kwargs)

In [ ]:
#@title Encode-Process-Decode Architecture { form-width: "30%" }

class EncodeProcessDecode(snt.AbstractModule):
    """Full encode-process-decode model.

      The model we explore includes three components:
      - An "Encoder" graph net, which independently encodes the edge, node, and
        global attributes (does not compute relations etc.).
      - A "Core" graph net, which performs N rounds of processing (message-passing)
        steps. The input to the Core is the concatenation of the Encoder's output
        and the previous output of the Core (labeled "Hidden(t)" below, where "t" is
        the processing step).
      - A "Decoder" graph net, which independently decodes the edge, node, and
        global attributes (does not compute relations etc.), on each message-passing
        step.
        
                            h(t)        h(t + 1)
                *---------*  |  *------*    |  *---------*    *---------*
                |         |  |  |      |    |  |         |    |         |
      Input --->| Encoder |  *->| Core |----*->| Decoder |--->| Lookup  |--->Output(t)
                |         |---->|      |       |         |    |         |
                *---------*     *------*       *---------*    *---------*
    """

    def __init__(self, edge_output_size, name="EncodeProcessDecode"):
        super(EncodeProcessDecode, self).__init__(name=name)
        with self._enter_variable_scope():
            self._encoder = MLPGraphIndependent()
            self._core = GraphGatedNonLocalNetwork()
            #self._decoder = MLPGraphIndependent()
            self._lookup = LocalRoutingNetwork(edge_output_size)

    def _build(self, input_op, num_processing_steps, is_training):
        latent0 = self._encoder(input_op, is_training=is_training)
        latent = latent0
        output_ops = []
        node_batch_size = tf.reduce_sum(latent.n_node)
        edge_batch_size = tf.reduce_sum(latent.n_edge)
        self._core.reset_state(edge_batch_size, node_batch_size)
        for _ in range(num_processing_steps):
            core_input = utils_tf.concat([latent0, latent], axis=1, use_global=False)
            latent = self._core(core_input, is_training=is_training)
            #decoded_op = self._decoder(latent, is_training=is_training)
            output_ops.append(self._lookup(latent, is_training=is_training))
        return output_ops

#  Set up model training and evaluation

The model we explore includes three components:
- An "Encoder" graph net, which independently encodes the edge, node, and
    global attributes (does not compute relations etc.).
- A "Core" graph net, which performs N rounds of processing (message-passing)
    steps. The input to the Core is the concatenation of the Encoder's output
    and the previous output of the Core. Moreover the core uses a recurrent layer to
    process the node features.
- A "Decoder" graph net, which decodes the edge attributes to the output shape,
    which is the dimension of the hot-one vector that represents with a edge (link) is
    used to achieve a previous determined end node. This computation is made on each
    message-passing step.
        
The model is trained by supervised learning. Input graphs are procedurally generated, and output
graphs have the same structure with the edges of the shortest path labeled (using 2-element 1-hot
vectors).

The training loss is computed on the output of each processing step. The reason for this is to
encourage the model to try to solve the problem in as few steps as possible. It also helps make
the output of intermediate steps more interpretable.

There's no need for a separate evaluate dataset because the inputs are never repeated, so the training
loss is the measure of performance on graphs from the input distribution.

We also evaluate how well the models generalize to graphs which are up to twice as large as those on which
it was trained. The loss is computed only on the final processing step.

Variables with the suffix _tr are training parameters, and variables with the suffix _ge are test/generalization
parameters.

In [ ]:
#@title Set up model { form-width: "30%" }

tf.set_random_seed(SEED)
tf.reset_default_graph()

random_state = np.random.RandomState(seed=SEED)

# Model parameters.
# Number of processing (message-passing) steps.
num_processing_steps_tr = 20 #@param{type: 'integer'}
num_processing_steps_ge = 20 #@param{type: 'integer'}

# Data / training parameters.
num_training_iterations = 50000 #@param{type: 'integer'}

batch_size_tr = 32 #@param{type: 'integer'}
batch_size_ge = 32 #@param{type: 'integer'}

# Number of nodes per graph sampled uniformly from this range.
min_num_nodes_tr = 8 #@param{type: 'integer'}
max_num_nodes_tr = 20 #@param{type: 'integer'}
min_num_nodes_ge = 16 #@param{type: 'integer'}
max_num_nodes_ge = 33 #@param{type: 'integer'}
num_nodes_min_max_tr = (min_num_nodes_tr, max_num_nodes_tr)
num_nodes_min_max_ge = (min_num_nodes_ge, max_num_nodes_ge)

batch_generator_tr = pybrite.graph_batch_generator(
    batch_size_tr, num_nodes_min_max_tr, random_state=random_state)#, input_fields=dict(node=("pos",)), global_field="pos")
batch_generator_ge = pybrite.graph_batch_generator(
    batch_size_ge, num_nodes_min_max_ge, random_state=random_state)#, input_fields=dict(node=("pos",)), global_field="pos")

# Data.
# Input and target placeholders.
input_ph, target_ph, weight_ph, is_training_ph = create_placeholders(batch_generator_tr)

# Connect the data to the model.
# Instantiate the model.
model = EncodeProcessDecode(edge_output_size=2)
# A list of outputs, one per processing step.
output_ops_tr = model(input_ph, num_processing_steps_tr, is_training_ph)
output_ops_ge = model(input_ph, num_processing_steps_ge, is_training_ph)

# Training loss.
loss_ops_tr = create_loss_ops(target_ph, output_ops_tr, weight_ph)
# Loss across processing steps.
loss_op_tr = sum(loss_ops_tr) / num_processing_steps_tr
# Test/generalization loss.
loss_ops_ge = create_loss_ops(target_ph, output_ops_ge, weight_ph)
loss_op_ge = loss_ops_ge[-1]  # Loss from final processing step.

# Optimizer.
## Fixed
#learning_rate = 1e-3
#optimizer = tf.train.AdamOptimizer(learning_rate)
#step_op = optimizer.minimize(loss_op_tr)
## Dynamically TF Way
starter_learning_rate = 1e-2
global_step = tf.Variable(0, trainable=False)
#learning_rate = tf.train.cosine_decay_restarts(starter_learning_rate, global_step, first_decay_steps=1000, m_mul=0.99, alpha=5e-6)
learning_rate = tf.train.polynomial_decay(starter_learning_rate, global_step, decay_steps=15000, end_learning_rate=1e-4, power=3)
optimizer = tf.train.AdamOptimizer(learning_rate)#tf.train.MomentumOptimizer(learning_rate, momentum=0.6)
step_op = optimizer.minimize(loss_op_tr, global_step=global_step)

# Lets an iterable of TF graphs be output from a session as NP graphs.
input_ph, target_ph = make_all_runnable_in_session(input_ph, target_ph)

saver = tf.train.Saver()

In [ ]:
#@title Reset session  { form-width: "30%" }

# This cell resets the Tensorflow session, but keeps the same computational
# graph.

try:
    sess.close()
except NameError:
    pass
sess = tf.Session()
sess.run(tf.global_variables_initializer())

last_iteration = 18000 #@param{type: 'integer'}
logged_iterations = []
losses_tr = []
corrects_tr = []
solveds_tr = []
losses_ge = []
corrects_ge = []
solveds_ge = []

restore_path = os.path.join(DRIVE_PATH, "TF-GNN-W-Sess/Sess %s/"%last_iteration)
if os.path.isdir(restore_path):
    saver.restore(sess, os.path.join(restore_path, "dm.ckpt"))

In [ ]:
#@title Run training  { form-width: "30%" }

# You can interrupt this cell's training loop at any time, and visualize the
# intermediate results by running the next cell (below). You can then resume
# training by simply executing this cell again.

# How much time between logging and printing the current results.
log_every_seconds = 20

print("Iteration,Elapsed Time (s),Loss Tr,Loss Ge,"
      "Accuracy Tr,Solved Tr,Accuracy Ge,Solved Ge,"
      "True Accuracy Tr,True Solved Tr,True Accuracy Ge,True Solved Ge,"
      "False Accuracy Tr,False Solved Tr,False Accuracy Ge,False Solved Ge,"
      "Avg Batch Max Dist Diff Tr,Avg Batch Min Dist Diff Tr,Avg Batch Avg Dist Diff Tr,"
      "Avg Batch Max Hops Diff Tr,Avg Batch Min Hops Diff Tr,Avg Batch Avg Hops Diff Tr,"
      "Max Batch unreachable Tr,Min Batch unreachable Tr,Avg Batch unreachable Tr,"
      "Avg Batch Max Dist Diff Ge,Avg Batch Min Dist Diff Ge,Avg Batch Avg Dist Diff Ge,"
      "Avg Batch Max Hops Diff Ge,Avg Batch Min Hops Diff Ge,Avg Batch Avg Hops Diff Ge,"
      "Max Batch unreachable Ge,Min Batch unreachable Ge,Avg Batch unreachable Ge")

start_time = time.time()
last_log_time = start_time

for iteration in range(last_iteration, num_training_iterations):
    last_iteration = iteration
    feed_dict, _ = create_feed_dict(batch_generator_tr, True, [.3, 1], input_ph, target_ph, is_training_ph, weight_ph)
    train_values = sess.run({
        "step": step_op,
        "input": input_ph,
        "target": target_ph,
        "loss": loss_op_tr,
        "outputs": output_ops_tr
    },
        feed_dict=feed_dict)
    
    the_time = time.time()
    elapsed_since_last_log = the_time - last_log_time
    if (elapsed_since_last_log > log_every_seconds) or ((iteration + 1) % 500 == 0):
        last_log_time = the_time
        feed_dict, _ = create_feed_dict(batch_generator_ge, False, [.3, 1], input_ph, target_ph, is_training_ph, weight_ph)
        test_values = sess.run({
            "input": input_ph,
            "target": target_ph,
            "loss": loss_op_ge,
            "outputs": output_ops_ge
        },
            feed_dict=feed_dict)
        
        tr_path_metrics = aggregator_path_metrics(train_values["input"], train_values["target"], train_values["outputs"][-1])
        ge_path_metrics = aggregator_path_metrics(test_values["input"], test_values["target"], test_values["outputs"][-1])

        correct_tr, solved_tr, true_correct_tr, true_solved_tr, false_correct_tr, false_solved_tr = compute_accuracy(
            train_values["target"], train_values["outputs"][-1])
        correct_ge, solved_ge, true_correct_ge, true_solved_ge, false_correct_ge, false_solved_ge = compute_accuracy(
            test_values["target"], test_values["outputs"][-1])
        
        elapsed = time.time() - start_time
        losses_tr.append(train_values["loss"])
        corrects_tr.append(correct_tr)
        solveds_tr.append(solved_tr)
        losses_ge.append(test_values["loss"])
        corrects_ge.append(correct_ge)
        solveds_ge.append(solved_ge)
        logged_iterations.append(iteration)
        if (iteration + 1) % 500 == 0:
            sess_path = os.path.join(DRIVE_PATH, "TF-GNN-W-Sess/Sess %s/"%(iteration + 1))
            if not os.path.isdir(sess_path):
                os.makedirs(sess_path)
            _ = saver.save(sess, os.path.join(sess_path, "dm.ckpt"))
        print("{:05d}, {:.1f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, "
              "{:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, "
              "{:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}, {:.4f}".format(
                  iteration, elapsed, train_values["loss"], test_values["loss"], correct_tr, solved_tr, correct_ge, solved_ge,
                  true_correct_tr, true_solved_tr, true_correct_ge, true_solved_ge,
                  false_correct_tr, false_solved_tr, false_correct_ge, false_solved_ge,
                  tr_path_metrics["avg_batch_max_dist_diff"], tr_path_metrics["avg_batch_min_dist_diff"], tr_path_metrics["avg_batch_avg_dist_diff"],
                  tr_path_metrics["avg_batch_max_hops_diff"], tr_path_metrics["avg_batch_min_hops_diff"], tr_path_metrics["avg_batch_avg_hops_diff"],
                  tr_path_metrics["max_batch_unreachable_p"], tr_path_metrics["min_batch_unreachable_p"], tr_path_metrics["avg_batch_unreachable_p"],
                  ge_path_metrics["avg_batch_max_dist_diff"], ge_path_metrics["avg_batch_min_dist_diff"], ge_path_metrics["avg_batch_avg_dist_diff"],
                  ge_path_metrics["avg_batch_max_hops_diff"], ge_path_metrics["avg_batch_min_hops_diff"], ge_path_metrics["avg_batch_avg_hops_diff"],
                  ge_path_metrics["max_batch_unreachable_p"], ge_path_metrics["min_batch_unreachable_p"], ge_path_metrics["avg_batch_unreachable_p"]))

In [ ]:
#@title Set Environment to See Distribution{ form-width: "30%" }
#@markdown <br>Some parameters in the TF sesseion will be redefined to take measures of the path metrics in a batch.

tf.set_random_seed(SEED)
tf.res2et_default_graph()

random_state = np.random.RandomState(seed=SEED)

# Model parameters.
# Number of processing (message-passing) steps.
num_processing_steps_tr = 30 #@param{type: 'integer'}
num_processing_steps_ge = 50 #@param{type: 'integer'}

batch_size_tr = 1000 #@param{type: 'integer'}
batch_size_ge = 1000 #@param{type: 'integer'}

# Number of nodes per graph sampled uniformly from this range.
min_num_nodes_tr = 15 #@param{type: 'integer'}
max_num_nodes_tr = 30 #@param{type: 'integer'}
min_num_nodes_ge = 25 #@param{type: 'integer'}
max_num_nodes_ge = 50 #@param{type: 'integer'}
num_nodes_min_max_tr = (min_num_nodes_tr, max_num_nodes_tr)
num_nodes_min_max_ge = (min_num_nodes_ge, max_num_nodes_ge)

batch_generator_tr = pybrite.graph_batch_generator(
    batch_size_tr, num_nodes_min_max_tr, random_state=random_state)
batch_generator_ge = pybrite.graph_batch_generator(
    batch_size_ge, num_nodes_min_max_ge, random_state=random_state)

# Data.
# Input and target placeholders.
input_ph, target_ph = create_placeholders(batch_generator_tr)

# Connect the data to the model.
# Instantiate the model.
model = EncodeProcessDecode(edge_output_size=2)
# A list of outputs, one per processing step.
output_ops_tr = model(input_ph, num_processing_steps_tr)
output_ops_ge = model(input_ph, num_processing_steps_ge)

# Lets an iterable of TF graphs be output from a session as NP graphs.
input_ph, target_ph = make_all_runnable_in_session(input_ph, target_ph)

saver = tf.train.Saver()

In [ ]:
#@title Reset Session and Restore one from Drive{ form-width: "30%" }

try:
    sess.close()
except NameError:
    pass
sess = tf.Session()
#sess.run(tf.global_variables_initializer())

last_iteration = 9500 #@param{type: 'integer'}

restore_path = os.path.join(DRIVE_PATH, "TF-GNN-Sess/Sess %s/"%last_iteration)
try:
    saver.restore(sess, os.path.join(restore_path, "dm.ckpt"))
except:
    print("There are not any saved session for this last iteration.")
    raise

In [ ]:
feed_dict_tr = create_feed_dict(batch_generator_tr, input_ph, target_ph)
train_values = sess.run({
    "input": input_ph,
    "target": target_ph,
    "outputs": output_ops_tr
},
    feed_dict=feed_dict_tr)

feed_dict_ge = create_feed_dict(batch_generator_ge, input_ph, target_ph)
test_values = sess.run({
    "input": input_ph,
    "target": target_ph,
    "outputs": output_ops_ge
},
    feed_dict=feed_dict_ge)

In [ ]:
graph_idx = 0
hist_path = os.path.join(DRIVE_PATH, "Histograms/")
metrics_graph_generator = get_generator_path_metrics(train_values["input"], train_values["target"], train_values["outputs"][-1])
for diff_dist, diff_hops, unreachable_p in metrics_graph_generator:
    file_path = os.path.join(hist_path, "Tr/%s - %.3f.csv"%(graph_idx, unreachable_p))
    if not np.any(diff_dist == None):
        df = pd.DataFrame(np.array([diff_dist, diff_hops]).T, columns=["Distance Diff", "Hops Diff"])
        df.to_csv(file_path, index=False)
    graph_idx += 1

In [ ]:
graph_idx = 0
hist_path = os.path.join(DRIVE_PATH, "Histograms/")
metrics_graph_generator = get_generator_path_metrics(test_values["input"], test_values["target"], test_values["outputs"][-1])
for diff_dist, diff_hops, unreachable_p in metrics_graph_generator:
    file_path = os.path.join(hist_path, "Ge/%s - %.3f.csv"%(graph_idx, unreachable_p))
    if not np.any(diff_dist == None):
        df = pd.DataFrame(np.array([diff_dist, diff_hops]).T, columns=["Distance Diff", "Hops Diff"])
        df.to_csv(file_path, index=False)
    graph_idx += 1